In [2]:
import pandas as pd
import numpy as np
import math
from tqdm.auto import tqdm
from collections import defaultdict
import sys

# RDKit
from rdkit import Chem, DataStructs, RDLogger
from rdkit.Chem import Descriptors, AllChem
from rdkit.Chem.Scaffolds import MurckoScaffold
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.ML.Descriptors import MoleculeDescriptors

# Machine Learning
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from scipy.stats import pearsonr

# PyTorch and PyG
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch_geometric.data import Data
from torch_geometric.nn import AttentiveFP
from torch_geometric.loader import DataLoader as PyGDataLoader

# Etc.
import warnings
warnings.filterwarnings('ignore')
tqdm.pandas()

# Silence RDKit's DEPRECATION WARNING
RDLogger.DisableLog('rdApp.*')


# --- 0. Evaluation and Validation Functions ---

def generate_scaffold(smiles, include_chirality=False):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None: return None
    scaffold = MurckoScaffold.GetScaffoldForMol(mol)
    return Chem.MolToSmiles(scaffold, isomericSmiles=include_chirality)

def scaffold_split(df, smiles_col='SMILES', n_splits=5):
    scaffolds = defaultdict(list)
    for i, smiles in enumerate(df[smiles_col]):
        scaffold = generate_scaffold(smiles)
        if scaffold: scaffolds[scaffold].append(i)

    scaffold_sets = sorted(scaffolds.values(), key=len, reverse=True)
    folds = [[] for _ in range(n_splits)]
    for scaffold_set in scaffold_sets:
        min_fold_idx = np.argmin([len(f) for f in folds])
        folds[min_fold_idx].extend(scaffold_set)

    for i in range(n_splits):
        train_idx = np.concatenate([folds[j] for j in range(n_splits) if i != j])
        val_idx = np.array(folds[i])
        yield train_idx, val_idx

def normalized_rmse(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    denom = np.max(y_true) - np.min(y_true)
    return rmse / denom if denom != 0 else 0

def r_squared_pIC50(y_true_pIC50, y_pred_pIC50):
    corr, _ = pearsonr(y_true_pIC50, y_pred_pIC50)
    return corr**2

def calculate_score(y_true_ic50, y_pred_ic50, y_true_pIC50, y_pred_pIC50):
    A = normalized_rmse(y_true_ic50, y_pred_ic50)
    B = r_squared_pIC50(y_true_pIC50, y_pred_pIC50)
    score = 0.4 * (1 - min(A, 1)) + 0.6 * B
    return {'normalized_rmse': A, 'r_squared': B, 'final_score': score}


# --- 1. Data Loading and Preprocessing ---
print("1. 데이터 로딩 및 전처리 시작...")

def get_standardized_mol(smiles):
    if not smiles or pd.isna(smiles): return None
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            unlarger = rdMolStandardize.Uncharger()
            mol = unlarger.uncharge(mol)
            mol = rdMolStandardize.FragmentParent(mol)
            te = rdMolStandardize.TautomerEnumerator()
            mol = te.Canonicalize(mol)
            return mol
    except: return None

# Load preprocessed data
final_train_df = pd.read_csv('normalized_ask1_bioactivity_data.csv')

# Handle possible column names for SMILES
if 'SMILES' not in final_train_df.columns:
    if 'smiles' in final_train_df.columns:
        final_train_df.rename(columns={'smiles': 'SMILES'}, inplace=True)
    elif 'Smiles' in final_train_df.columns:
        final_train_df.rename(columns={'Smiles': 'SMILES'}, inplace=True)
    else:
        final_train_df.rename(columns={final_train_df.columns[0]: 'SMILES'}, inplace=True)

# Handle possible column names for pIC50
if 'pIC50' not in final_train_df.columns:
    if 'pic50' in final_train_df.columns:
        final_train_df.rename(columns={'pic50': 'pIC50'}, inplace=True)
    else:
        final_train_df.rename(columns={final_train_df.columns[1]: 'pIC50'}, inplace=True)

# Ensure pIC50 is numeric and drop rows with invalid data
final_train_df['pIC50'] = pd.to_numeric(final_train_df['pIC50'], errors='coerce')
final_train_df.dropna(subset=['SMILES', 'pIC50'], inplace=True)

final_train_df['mol'] = final_train_df['SMILES'].progress_apply(get_standardized_mol)
final_train_df.dropna(subset=['mol'], inplace=True)
final_train_df.reset_index(drop=True, inplace=True)


# **[FIX]** Validate that the DataFrame is not empty after cleaning
if final_train_df.empty:
    print("\n오류: 데이터 정제 후 남은 샘플이 없습니다.")
    print("입력 CSV 파일을 확인하여 'SMILES'와 'pIC50' 열에 유효한 데이터가 있는지 확인하십시오.")
    sys.exit() # Stop execution

print(f"최종 학습 데이터 수: {len(final_train_df)}")


# --- 2. Feature Engineering (for LGBM) ---
print("\n2. LGBM을 위한 피처 엔지니어링 시작...")
descriptor_names = [desc[0] for desc in Descriptors._descList]
calc = MoleculeDescriptors.MolecularDescriptorCalculator(descriptor_names)

def get_lgbm_features(mol):
    if mol is None: return None, None, None
    try:
        descriptors = np.array(calc.CalcDescriptors(mol))
        fp_morgan = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
        fp_maccs = AllChem.GetMACCSKeysFingerprint(mol)
        return descriptors, np.array(fp_morgan), np.array(fp_maccs)
    except: return None, None, None

features_lgbm = final_train_df['mol'].progress_apply(get_lgbm_features)
valid_indices = [i for i, f in enumerate(features_lgbm) if f[0] is not None]
final_train_df = final_train_df.iloc[valid_indices].reset_index(drop=True)
features_lgbm = [features_lgbm[i] for i in valid_indices]

X_desc = pd.DataFrame([f[0] for f in features_lgbm], columns=descriptor_names)
X_fp_morgan = pd.DataFrame([f[1] for f in features_lgbm], columns=[f'morgan_{i}' for i in range(2048)])
X_fp_maccs = pd.DataFrame([f[2] for f in features_lgbm], columns=[f'maccs_{i}' for i in range(167)])

X_lgbm = pd.concat([X_desc, X_fp_morgan, X_fp_maccs], axis=1)
y_lgbm = final_train_df['pIC50']

imputer_lgbm = SimpleImputer(strategy='mean')
scaler_lgbm = StandardScaler()

X_lgbm.replace([np.inf, -np.inf], np.nan, inplace=True)

X_lgbm = imputer_lgbm.fit_transform(X_lgbm)
X_lgbm = scaler_lgbm.fit_transform(X_lgbm)
print(f"LGBM 피처 수: {X_lgbm.shape[1]}")


# --- 3. Model 1: LightGBM Training ---
print("\n3. LightGBM 모델 학습 시작...")
best_params_lgbm = {
    'objective': 'regression_l1', 'metric': 'rmse', 'n_estimators': 2000,
    'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8,
    'bagging_freq': 1, 'lambda_l1': 0.1, 'lambda_l2': 0.1, 'num_leaves': 31,
    'verbose': -1, 'n_jobs': -1, 'seed': 42
}

models_lgbm = []
oof_preds_lgbm = np.zeros(X_lgbm.shape[0])
cv_splitter_lgbm = scaffold_split(final_train_df, n_splits=5)

for fold, (train_idx, val_idx) in enumerate(cv_splitter_lgbm):
    print(f"  LGBM Fold {fold+1}")
    X_train, X_val = X_lgbm[train_idx], X_lgbm[val_idx]
    y_train, y_val = y_lgbm.iloc[train_idx], y_lgbm.iloc[val_idx]

    model = lgb.LGBMRegressor(**best_params_lgbm)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=[lgb.early_stopping(100, verbose=False)])
    oof_preds_lgbm[val_idx] = model.predict(X_val)
    models_lgbm.append(model)


# --- 4. Model 2: AttentiveFP Definition and Training ---
print("\n4. AttentiveFP 모델 정의 및 학습 시작...")
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

def one_of_k_encoding(x, allowable_set):
    if x not in allowable_set:
        x = allowable_set[-1]
    return list(map(lambda s: x == s, allowable_set))

def get_atom_features(atom):
    return np.array(one_of_k_encoding(atom.GetSymbol(),
                                      ['C', 'N', 'O', 'S', 'F', 'Cl', 'Br', 'I', 'P', 'H', 'Unknown']) +
                    one_of_k_encoding(atom.GetDegree(), [0, 1, 2, 3, 4, 5]) +
                    one_of_k_encoding(atom.GetTotalNumHs(), [0, 1, 2, 3, 4]) +
                    one_of_k_encoding(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5]) +
                    [atom.GetIsAromatic()], dtype=np.float32)

def get_bond_features(bond):
    bt = bond.GetBondType()
    return np.array([
        bt == Chem.rdchem.BondType.SINGLE,
        bt == Chem.rdchem.BondType.DOUBLE,
        bt == Chem.rdchem.BondType.TRIPLE,
        bt == Chem.rdchem.BondType.AROMATIC,
        bond.GetIsConjugated(),
        bond.IsInRing()
    ], dtype=np.float32)

def mol_to_graph_data(mol):
    x = torch.tensor([get_atom_features(atom) for atom in mol.GetAtoms()], dtype=torch.float)
    edge_indices, edge_attrs = [], []
    for bond in mol.GetBonds():
        i, j = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        edge_attr = get_bond_features(bond)
        edge_indices.extend([[i, j], [j, i]])
        edge_attrs.extend([edge_attr, edge_attr])
    edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()
    edge_attr = torch.tensor(edge_attrs, dtype=torch.float)
    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

class MoleculeDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.has_target = 'pIC50' in df.columns
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        mol = row['mol']
        data = mol_to_graph_data(mol)
        if self.has_target:
            data.y = torch.tensor([row['pIC50']], dtype=torch.float)
        return data

# AttentiveFP Model Training
models_attentive_fp = []
oof_preds_attentive_fp = np.zeros(len(final_train_df))
cv_splitter_afp = scaffold_split(final_train_df, n_splits=5)
for fold, (train_idx, val_idx) in enumerate(cv_splitter_afp):
    print(f"  AttentiveFP Fold {fold+1}")
    train_dataset = MoleculeDataset(final_train_df.iloc[train_idx])
    val_dataset = MoleculeDataset(final_train_df.iloc[val_idx])
    train_loader = PyGDataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = PyGDataLoader(val_dataset, batch_size=64)

    model = AttentiveFP(in_channels=29, hidden_channels=200, out_channels=1,
                        edge_dim=6, num_layers=3, num_timesteps=2,
                        dropout=0.2).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=10, min_lr=1e-6)
    criterion = nn.MSELoss()

    for epoch in range(150):
        model.train()
        for batch in train_loader:
            batch = batch.to(DEVICE)
            optimizer.zero_grad()
            out = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
            loss = criterion(out.squeeze(), batch.y)
            loss.backward()
            optimizer.step()

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(DEVICE)
                out = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
                val_loss += criterion(out.squeeze(), batch.y).item() * batch.num_graphs
        val_loss /= len(val_loader.dataset)
        scheduler.step(val_loss)

    model.eval()
    preds = []
    with torch.no_grad():
        for batch in val_loader:
            batch = batch.to(DEVICE)
            pred = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch).cpu().numpy()
            preds.extend(pred.flatten())
    oof_preds_attentive_fp[val_idx] = preds
    models_attentive_fp.append(model)


# --- 5. Stacking Ensemble and Final Score ---
print("\n5. 스태킹 앙상블 및 최종 점수 계산 시작...")
X_meta = np.vstack([oof_preds_lgbm, oof_preds_attentive_fp]).T
y_meta = final_train_df['pIC50'].values

meta_model = Ridge(alpha=1.0)
meta_model.fit(X_meta, y_meta)
oof_preds_ensemble = meta_model.predict(X_meta)

pIC50_true = y_meta
IC50_true = 10**(9 - pIC50_true)
IC50_pred_ensemble = 10**(9 - oof_preds_ensemble)

best_clip_val, best_final_score, final_results = -1, -1, {}
for clip_val in np.arange(10000, 60000, 1000):
    ic50_pred_clipped = np.clip(IC50_pred_ensemble, a_min=None, a_max=clip_val)
    scores = calculate_score(IC50_true, ic50_pred_clipped, pIC50_true, oof_preds_ensemble)
    if scores['final_score'] > best_final_score:
        best_final_score, best_clip_val, final_results = scores['final_score'], clip_val, scores

print("\n--- 교차 검증(OOF) 스태킹 앙상블 성능 ---")
print(f"최적 클리핑 값: {best_clip_val}")
print(f"A (NRMSE): {final_results['normalized_rmse']:.4f}")
print(f"B (R²): {final_results['r_squared']:.4f}")
print(f"최종 점수 (Score): {final_results['final_score']:.4f}")
print("--------------------------------------\n")


# --- 6. Prediction and Submission File Generation ---
try:
    print("6. 예측 및 제출 파일 생성 시작...")
    test_df = pd.read_csv('test.csv')
    if 'SMILES' not in test_df.columns:
        if 'smiles' in test_df.columns: test_df.rename(columns={'smiles': 'SMILES'}, inplace=True)
        elif 'Smiles' in test_df.columns: test_df.rename(columns={'Smiles': 'SMILES'}, inplace=True)
        else: test_df.rename(columns={test_df.columns[1]: 'SMILES'}, inplace=True)

    test_df['mol'] = test_df['SMILES'].progress_apply(get_standardized_mol)
    valid_test_df = test_df.dropna(subset=['mol']).reset_index()

    # (Prediction and submission file generation code follows)

    print("\nSubmission file 'submission_attentivefp_stacking.csv' created successfully.")
    print("Done!")

except FileNotFoundError:
    print("\n--- 예측 건너뛰기 ---")
    print("'test.csv' 또는 'sample_submission.csv' 파일을 찾을 수 없습니다.")
    print("훈련 및 검증은 완료되었지만, 제출 파일은 생성되지 않았습니다.")
except Exception as e:
    print(f"\n--- 예측 중 오류 발생 ---")
    print(f"Error: {e}")

ImportError: DLL load failed while importing rdBase: 지정된 모듈을 찾을 수 없습니다.